In [1]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import vstack
import twitter_preproc
import importlib
importlib.reload(twitter_preproc)
from twitter_preproc import *
from operator import attrgetter
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from twitter_preproc import twitter_preproc

from pyspark.ml.feature import OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

import torch.nn as nn
import torch
import torch.nn.functional as F
import math
import torch.optim as optim

import sys, traceback

import NNPreprocessor as NNP
importlib.reload(NNP)
import NNCFNet
importlib.reload(NNCFNet)

<module 'NNCFNet' from '/home/adbs20/e1447920/recsys-twitter/src/NNCFNet.py'>

In [2]:
# Building Spark Context
# conf = SparkConf().setAll([('spark.executor.memory', '32g'), ('spark.executor.instances','8'),('spark.executor.cores', '12'), ('spark.driver.memory','64g'), ('spark.driver.memoryOverhead', '64g')])
conf = SparkConf()
spark = SparkSession.builder.appName("nncf_eval").config(conf=conf).getOrCreate()
sc = spark.sparkContext

# LOAD DATA 

In [3]:
%%time

train = "///tmp/traintweet_1000.tsv" # irrelevant; only needed for preproc class to shut up
#test = "///user/e11920598/test_1000.tsv" # no access!
test = "///tmp/traintweet_1000.tsv" # nat actually train set
#test= "///user/pknees/RSC20/test.tsv"

preproc = twitter_preproc(spark, sc, train, testFile=test, MF=True)

CPU times: user 11.8 ms, sys: 6.98 ms, total: 18.7 ms
Wall time: 2.13 s


In [4]:
%%time

test_df = preproc.getTestDF()

CPU times: user 7 µs, sys: 2 µs, total: 9 µs
Wall time: 12.2 µs


In [ ]:
%%time

nnpre = NNP.NNPreprocessor()
users, tweets, target = nnpre.nn_preprocess(test_df)

# LOAD MODEL

In [ ]:
model_PATH = '../misc/NNCF_model_save.pth'
users_dim = 9999
tweets_dim = 9999
k = 32

model = NNCFNet.Net(users_dim, tweets_dim, k)
model.load_state_dict(torch.load(model_PATH))
model.eval()

# PREDICT TEST DATA

In [ ]:
# padding
padding_users = users_dim - users.shape[1] - 1
padding_tweets = tweets_dim - tweets.shape[1] - 1
padded_users = F.pad(users, (padding,1))
padded_tweets = F.pad(tweets, (padding,1))

# run
outputs = model(padded_users, padded_tweets)

# label classes
probabilities = outputs.detach().numpy().flatten()
predicted = probabilities > probabilities.mean()
truth = target.detach().numpy().flatten().astype(bool)

# EVALUATE TEST DATA

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(truth, predicted)

# EXPORT

In [ ]:
pass